# Imbalanced Data Join

## Background

We've run into some applied situations in the wild where joins on highly imbalanced data behaves poorly in Dask. In some situtions the run time is more than desired, and in others workers may die unexpectedly. The purpose of this notebook is to create two simulated data sets that exhibits the type of problematic imbalance that we've encountred in the wild. We'll have a few parameters in here so we can create datasets of different sizes that exhibit the gnarly imbalance and these can be used for practical test cases. 



In [3]:
import pandas as pd
import scipy.stats as ss

## df1 - data with large record count

The first dataframe is going to be  `df1` and it is the larger table in terms of record count. The fields in `df1` are as follows:

`key`: the key used to join with `df2` later. The number of unique keys is low compared to the number of records in each key

`bucket1`: Every key has the same number of unique values in `bucket1`. That number is set by a constant below

`group1`: There are 4 groupings. Each of these groupings is assinged a key and they are later used in a group by 

`group2`: see above

`group3`: see above

`group4`: see above

`value`: This is the value we will sum later. It's random and has no impact on the process in any way. 